除模型评估之外，在深入研究模型开发之前，我们还必须解决另一个重要问题：将数据输入神经网络之前，如何准备输入数据和目标？许多数据预处理方法和特征工程技术都是和特定领域相关的（比如只和文本数据或图像数据相关），我们将在后续章节的实例中介绍这些内容。现在我们要介绍所有数据领域通用的基本方法。

### 4.3.1　神经网络的数据预处理
数据预处理的目的是使原始数据更适于用神经网络处理，包括向量化、标准化、处理缺失值和特征提取。

#### 4.3.1.1 向量化
神经网络的所有输入和目标都必须是浮点数张量（在特定情况下可以是整数张量）。无论处理什么数据（声音、图像还是文本），都必须首先将其转换为张量，这一步叫作数据向量化（data vectorization）。   

例如，在前面两个文本分类的例子中，开始时文本都表示为整数列表（代表单词序列），然后我们用 one-hot编码将其转换为float32格式的张量。在手写数字分类和预测房价的例子中，数据已经是向量形式，所以可以跳过这一步。

#### 4.3.1.2 值标准化

在手写数字分类的例子中，开始时图像数据被编码为0-255范围内的整数，表示灰度值。将这一数据输入网络之前，你需要将其转换为float32格式并除以  255，这样就得到0-1范围内的浮点数。同样，预测房价时，开始时特征有各种不同的取值范围，有些特征是较小的浮点数，有些特征是相对较大的整数。将这一数据输入网络之前，你需要对每个特征分别做标准化，使其均值为 0、标准差为 1。

一般来说，将取值相对较大的数据（比如多位整数，比网络权重的初始值大很多）或异质数据（heterogeneous data，比如数据的一个特征在0-1范围内，另一个特征在100-200范围内）输入到神经网络中是不安全的。这么做可能导致较大的梯度更新，进而导致网络无法收敛。为了让网络的学习变得更容易，输入数据应该具有以下特征。  

1. 取值较小：大部分值都应该在0~1范围内。
2. 同质性（homogenous）：所有特征的取值都应该在大致相同的范围内。  

此外，下面这种更严格的标准化方法也很常见，而且很有用，虽然不一定总是必需的（例如，对于数字分类问题就不需要这么做）。

1. 将每个特征分别标准化，使其平均值为0。
2. 将每个特征分别标准化，使其标准差为1。

In [ ]:
x -= x.mean(axis=0)
x /= x.std(axis=0)

#### 4.3.1.3 处理缺失值
你的数据中有时可能会有缺失值。例如在房价的例子中，第一个特征（数据中索引编号为0的列）是人均犯罪率。如果不是所有样本都具有这个特征的话，怎么办？那样你的训练数据或测试数据将会有缺失值。  

一般来说，对于神经网络，将缺失值设置为0是安全的，只要0不是一个有意义的值。网络能够从数据中学到0意味着缺失数据，并且会忽略这个值。  

注意，如果测试数据中可能有缺失值，而网络是在没有缺失值的数据上训练的，那么网络不可能学会忽略缺失值。在这种情况下，你应该人为生成一些有缺失项的训练样本：多次复制一些训练样本，然后删除测试数据中可能缺失的某些特征。

### 4.3.2　特征工程

特征工程（feature  engineering）是指将数据输入模型之前，利用你自己关于数据和机器学习算法（这里指神经网络）的知识对数据进行硬编码的变换（不是模型学到的），以改善模型的效果。多数情况下，一个机器学习模型无法从完全任意的数据中进行学习。呈现给模型的数据应该便于模型进行学习。

假设你想开发一个模型，输入一个时钟图像，模型能够输出对应的时间  

如果你选择用图像的原始像素作为输入数据，那么这个机器学习问题将非常困难。你需要用卷积神经网络来解决这个问题，而且还需要花费大量的计算资源来训练网络。

但如果你从更高的层次理解了这个问题（你知道人们怎么看时钟上的时间），那么可以为机器学习算法找到更好的输入特征，比如你可以编写  5行  Python脚本，找到时钟指针对应的黑色像素并输出每个指针尖的 (x, y)坐标，这很简单。然后，一个简单的机器学习算法就可以学会这些坐标与时间的对应关系。

你还可以进一步思考：进行坐标变换，将 (x, y)坐标转换为相对于图像中心的极坐标。这样输入就变成了每个时钟指针的角度 theta。现在的特征使问题变得非常简单，根本不需要机器学习，因为简单的舍入运算和字典查找就足以给出大致的时间。

这就是特征工程的本质：用更简单的方式表述问题，从而使问题变得更容易。它通常需要深入理解问题。  

例如，卷积神经网络在 MNIST数字分类问题上取得成功之前，其解决方法通常是基于硬编码的特征，比如数字图像中的圆圈个数、图像中每个数字的高度、像素值的直方图等。   

1. 良好的特征仍然可以让你用更少的资源更优雅地解决问题。例如，使用卷积神经网络来读取钟面上的时间是非常可笑的。
2. 良好的特征可以让你用更少的数据解决问题。深度学习模型自主学习特征的能力依赖于大量的训练数据。如果只有很少的样本，那么特征的信息价值就变得非常重要